In [1]:
# Cài đặt thư viện cần thiết cho AI (nếu chưa có)
# %pip install pandas numpy scikit-learn pyarrow

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
import warnings

# Tắt cảnh báo đỏ
warnings.filterwarnings('ignore')

print("✅ Đã nạp xong thư viện AI!")

✅ Đã nạp xong thư viện AI!


In [2]:
def load_data_optimized(file_path):
    print(f"🔄 Đang đọc file '{file_path}'...")
    
    # Định nghĩa kiểu dữ liệu để tăng tốc
    column_types = {
        'manufacturer': 'string',
        'model': 'string',
        'year': 'int32',
        'mileage': 'float32',
        'price': 'float32',
        'one_owner': 'float32',         # 1.0 hoặc 0.0
        'accidents_or_damage': 'float32', # 1.0 hoặc 0.0
        'driver_rating': 'float32',
        'price_drop': 'float32',
        'mpg': 'string' 
    }
    
    # Chỉ đọc các cột liên quan đến Rủi ro
    cols_to_use = list(column_types.keys())

    try:
        # Ưu tiên engine pyarrow
        df = pd.read_csv(file_path, usecols=cols_to_use, dtype=column_types, engine='pyarrow')
    except:
        try:
            # Fallback nếu không có pyarrow
            df = pd.read_csv(file_path, usecols=cols_to_use, dtype=column_types)
        except FileNotFoundError:
            print("⚠️ Không thấy file. Đang tạo dữ liệu giả lập để demo...")
            # Dữ liệu giả lập cho demo
            return pd.DataFrame({
                'manufacturer': ['Toyota', 'Ford', 'BMW', 'Honda', 'Audi'] * 200,
                'year': [2015, 2010, 2018, 2012, 2020] * 200,
                'mileage': [50000, 150000, 30000, 120000, 10000] * 200,
                'accidents_or_damage': [0, 1, 0, 1, 0] * 200,
                'one_owner': [1, 0, 1, 0, 1] * 200,
                'driver_rating': [4.8, 3.5, 4.5, 4.0, 5.0] * 200,
                'price_drop': [0, 500, 0, 1000, 0] * 200,
                'mpg': ['30-35'] * 1000,
                'price': [20000] * 1000
            })

    print(f"✅ Đã tải thành công {len(df):,} dòng dữ liệu.")
    return df

# Thay 'cars.csv' bằng file của bạn
df_raw = load_data_optimized('cars.csv')

🔄 Đang đọc file 'cars.csv'...
✅ Đã tải thành công 762,091 dòng dữ liệu.


In [3]:
def preprocess_and_label_risk(df):
    df = df.copy()
    
    # 1. Xử lý dữ liệu thiếu (Fill NaN)
    # Tai nạn: Nếu thiếu coi như không (0)
    df['accidents_or_damage'] = df['accidents_or_damage'].fillna(0)
    # Chủ xe: Nếu thiếu coi như nhiều chủ (0)
    df['one_owner'] = df['one_owner'].fillna(0)
    # Rating: Nếu thiếu lấy trung bình
    df['driver_rating'] = df['driver_rating'].fillna(df['driver_rating'].mean())
    # Mileage: Nếu thiếu lấy trung bình
    df['mileage'] = df['mileage'].fillna(df['mileage'].mean())

    # 2. TẠO NHÃN RỦI RO (RISK LABELING)
    # Logic chuyên gia để gán nhãn cho dữ liệu huấn luyện:
    # Điểm càng cao -> Rủi ro càng lớn
    
    risk_scores = []
    
    for i, row in df.iterrows():
        score = 0
        
        # - Có tai nạn: +3 điểm (Rất nguy hiểm)
        if row['accidents_or_damage'] == 1:
            score += 3
            
        # - Đời xe quá sâu (trước 2015): +1 điểm
        if row['year'] < 2015:
            score += 1
            
        # - ODO quá cao (>150k dặm): +2 điểm, (>100k): +1 điểm
        if row['mileage'] > 150000:
            score += 2
        elif row['mileage'] > 100000:
            score += 1
            
        # - Không phải chính chủ (qua nhiều đời chủ): +1 điểm
        if row['one_owner'] == 0:
            score += 1
            
        # - Đánh giá người lái thấp (<4.0): +1 điểm
        if row['driver_rating'] < 4.0:
            score += 1
            
        risk_scores.append(score)

    # Phân loại dựa trên tổng điểm
    # 0-1 điểm: Low (An toàn)
    # 2-3 điểm: Medium (Trung bình)
    # >3 điểm : High (Rủi ro cao)
    def classify_risk(s):
        if s <= 1: return 'Low'
        elif s <= 3: return 'Medium'
        else: return 'High'

    df['Risk_Level'] = [classify_risk(s) for s in risk_scores]
    
    print("✅ Đã tạo xong nhãn Rủi ro (Risk_Level) cho dữ liệu.")
    print("Phân bố rủi ro trong dataset:")
    print(df['Risk_Level'].value_counts())
    
    return df

# Áp dụng hàm
df_labeled = preprocess_and_label_risk(df_raw)

✅ Đã tạo xong nhãn Rủi ro (Risk_Level) cho dữ liệu.
Phân bố rủi ro trong dataset:
Risk_Level
Low       499187
Medium    138255
High      124649
Name: count, dtype: int64


In [4]:
print("🤖 ĐANG HUẤN LUYỆN MÔ HÌNH AI...")

# 1. Chọn đặc trưng (Features) để học
features = ['year', 'mileage', 'accidents_or_damage', 'one_owner', 'driver_rating', 'price_drop']
target = 'Risk_Level'

# 2. Chuẩn bị dữ liệu X (Input) và y (Output)
X = df_labeled[features]
y = df_labeled[target]

# 3. Chia tập Train (80%) và Test (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Khởi tạo và huấn luyện mô hình Random Forest
# Random Forest rất giỏi trong việc phân loại rủi ro có nhiều yếu tố hỗn hợp
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# 5. Đánh giá độ chính xác
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"\n✅ HUẤN LUYỆN HOÀN TẤT!")
print(f"Độ chính xác của mô hình: {accuracy * 100:.2f}%")
print("\nBáo cáo chi tiết:")
print(classification_report(y_test, y_pred))

🤖 ĐANG HUẤN LUYỆN MÔ HÌNH AI...

✅ HUẤN LUYỆN HOÀN TẤT!
Độ chính xác của mô hình: 100.00%

Báo cáo chi tiết:
              precision    recall  f1-score   support

        High       1.00      1.00      1.00     25006
         Low       1.00      1.00      1.00     99954
      Medium       1.00      1.00      1.00     27459

    accuracy                           1.00    152419
   macro avg       1.00      1.00      1.00    152419
weighted avg       1.00      1.00      1.00    152419



In [5]:
def predict_car_risk(year, mileage, accidents, one_owner, rating, price_drop):
    # Tạo dataframe từ thông số đầu vào
    input_data = pd.DataFrame({
        'year': [year],
        'mileage': [mileage],
        'accidents_or_damage': [accidents], # 1: Có, 0: Không
        'one_owner': [one_owner],           # 1: Có, 0: Không (Nhiều chủ)
        'driver_rating': [rating],
        'price_drop': [price_drop]
    })
    
    # AI dự đoán
    prediction = clf.predict(input_data)[0]
    
    # Lấy xác suất tự tin (AI chắc chắn bao nhiêu %)
    proba = clf.predict_proba(input_data).max() * 100
    
    # Tô màu cho đẹp
    color_map = {'Low': '🟢 THẤP (An toàn)', 'Medium': '🟡 TRUNG BÌNH', 'High': '🔴 CAO (Nguy hiểm)'}
    result_text = color_map.get(prediction, prediction)
    
    print("="*50)
    print(f"🔍 KẾT QUẢ DỰ ĐOÁN RỦI RO XE:")
    print(f"- Năm sx: {year} | ODO: {mileage:,} dặm")
    print(f"- Tai nạn: {'CÓ' if accidents else 'Không'} | Một chủ: {'Đúng' if one_owner else 'Sai'}")
    print("-" * 30)
    print(f"👉 MỨC ĐỘ RỦI RO: {result_text}")
    print(f"👉 Độ tin cậy của AI: {proba:.1f}%")
    print("="*50)

# --- THỬ NGHIỆM VỚI 3 TRƯỜNG HỢP ---

# Trường hợp 1: Xe ngon (Mới, 1 chủ, không tai nạn)
predict_car_risk(year=2020, mileage=20000, accidents=0, one_owner=1, rating=4.8, price_drop=0)

# Trường hợp 2: Xe bình thường (Cũ chút, đi nhiều, không tai nạn)
predict_car_risk(year=2014, mileage=120000, accidents=0, one_owner=0, rating=4.2, price_drop=500)

# Trường hợp 3: Xe nát (Tai nạn, đi nhiều)
predict_car_risk(year=2012, mileage=160000, accidents=1, one_owner=0, rating=3.5, price_drop=1000)

🔍 KẾT QUẢ DỰ ĐOÁN RỦI RO XE:
- Năm sx: 2020 | ODO: 20,000 dặm
- Tai nạn: Không | Một chủ: Đúng
------------------------------
👉 MỨC ĐỘ RỦI RO: 🟢 THẤP (An toàn)
👉 Độ tin cậy của AI: 100.0%
🔍 KẾT QUẢ DỰ ĐOÁN RỦI RO XE:
- Năm sx: 2014 | ODO: 120,000 dặm
- Tai nạn: Không | Một chủ: Sai
------------------------------
👉 MỨC ĐỘ RỦI RO: 🟡 TRUNG BÌNH
👉 Độ tin cậy của AI: 100.0%
🔍 KẾT QUẢ DỰ ĐOÁN RỦI RO XE:
- Năm sx: 2012 | ODO: 160,000 dặm
- Tai nạn: CÓ | Một chủ: Sai
------------------------------
👉 MỨC ĐỘ RỦI RO: 🔴 CAO (Nguy hiểm)
👉 Độ tin cậy của AI: 100.0%
